In [2]:
seed = 42

# Import libraries
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

import random
random.seed(seed)
from sklearn.utils import shuffle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.metrics import accuracy_score, f1_score, roc_curve, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from imblearn.over_sampling import SMOTE

from sklearn.feature_selection import mutual_info_classif
from scipy.spatial.distance import pdist, squareform
from scipy.stats import ttest_ind
import h5py

from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif, f_regression, SelectFromModel
sc = StandardScaler()

In [3]:

file_path = "/Users/alessiamenozzi/Desktop/ThesisPlaques/ThesisPlaques/CSV/data_rad_clin_DEF.csv"

data = pd.read_csv(file_path)
labels_column = data['label']
labels = labels_column.astype(int).tolist()

labels=np.array(labels)

# Estrazione dei numeri dai nomi dei pazienti
loaded_patients = data['IDs_new'].str.extract(r'(\d+)').astype(int).squeeze().tolist()

print("Labels:", labels)
print("Number of labels:", len(labels))
print("Patient Names: ", loaded_patients )

# Estrazione immagini
h5_file_path = '/Users/alessiamenozzi/Desktop/ThesisPlaques/ThesisPlaques/images_by_patient_final.h5'

loaded_class_images = []

with h5py.File(h5_file_path, 'r') as h5_file:
    for key in h5_file.keys():
        images = np.array(h5_file[key])
        loaded_class_images.append(images)


print("Lunghezza della lista dei batch di immagini:", len(loaded_class_images))
print("Shape del batch di immagini del primo paziente:", loaded_class_images[0].shape)
print("Shape del batch di immagini del secondo paziente:", loaded_class_images[1].shape)



Labels: [0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 1 1 0 1 1 0 1 0 0 0 1 1 1 0 1 0
 0 0 1 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 1 0 0
 0 1 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0
 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1]
Number of labels: 129
Patient Names:  [5, 12, 15, 16, 17, 19, 22, 23, 24, 25, 26, 27, 29, 30, 31, 33, 35, 36, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 50, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 64, 65, 68, 69, 70, 71, 74, 75, 76, 78, 79, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 123, 124, 126, 127, 128, 129, 133, 135, 136, 137, 138, 139, 141, 142, 144, 146, 147, 149, 150, 153, 155, 158, 159, 161, 163, 166, 168, 169, 170, 171, 175, 176, 178, 182, 183, 188, 189, 190, 193, 197, 199, 200, 205]
Lunghezza della lista dei batch di immagini: 129
Shape del batch di immagini del primo paziente: (28, 64, 64)
S

In [4]:
volume=[]
for i in range (0, len(loaded_class_images)):
    volume.append(np.dstack(loaded_class_images[i]))


In [8]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
import numpy as np

# Carica il modello pre-addestrato ResNet50, escludendo il top (strato fully connected)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Definisci una funzione per estrarre le caratteristiche da ciascuna slice
def extract_features_from_slice(slice_2d):
    slice_2d = np.expand_dims(slice_2d, axis=0)  # Aggiungi una dimensione per il batch
    features = base_model.predict(slice_2d, verbose=False)      # Estrai le caratteristiche
    return features


volumes_features=[]
# Iniziamo con un ciclo per processare tutti i volumi
for idx, vol in enumerate(volume):
    print(f"Processing volume {idx+1} with shape {vol.shape}")
    
    # Lista per contenere le caratteristiche di ogni slice del volume corrente
    feature_list = []
    
    # Esegui il loop su ogni slice del volume corrente e applica la funzione di estrazione delle caratteristiche
    for i in range(vol.shape[2]):  # Cicla sulla profondità del volume corrente
        slice_2d = vol[:, :, i]
        slice_2d = np.stack([slice_2d]*3, axis=-1)
        features = extract_features_from_slice(slice_2d)
        feature_list.append(features)
    
    volume_features = np.concatenate(feature_list, axis=-1)

    print(f'Volume {idx+1} features shape: {volume_features.shape}')
    volumes_features.append(volume_features)


Processing volume 1 with shape (64, 64, 28)
Volume 1 features shape: (1, 2, 2, 57344)
Processing volume 2 with shape (64, 64, 22)
Volume 2 features shape: (1, 2, 2, 45056)
Processing volume 3 with shape (64, 64, 29)
Volume 3 features shape: (1, 2, 2, 59392)
Processing volume 4 with shape (64, 64, 28)
Volume 4 features shape: (1, 2, 2, 57344)
Processing volume 5 with shape (64, 64, 30)
Volume 5 features shape: (1, 2, 2, 61440)
Processing volume 6 with shape (64, 64, 17)
Volume 6 features shape: (1, 2, 2, 34816)
Processing volume 7 with shape (64, 64, 25)
Volume 7 features shape: (1, 2, 2, 51200)
Processing volume 8 with shape (64, 64, 32)
Volume 8 features shape: (1, 2, 2, 65536)
Processing volume 9 with shape (64, 64, 37)
Volume 9 features shape: (1, 2, 2, 75776)
Processing volume 10 with shape (64, 64, 61)
Volume 10 features shape: (1, 2, 2, 124928)
Processing volume 11 with shape (64, 64, 30)
Volume 11 features shape: (1, 2, 2, 61440)
Processing volume 12 with shape (64, 64, 25)
Volu

In [10]:
# Converti la lista di caratteristiche in un array numpy
X = np.stack(volumes_features, axis=0)  # Forma: (num_volumes, features_per_slice)

y = np.array(labels)


ValueError: all input arrays must have the same shape

In [38]:
from tensorflow.keras import layers, models

# Definisci la forma di input
input_shape = X.shape[1:]  # Dovrebbe essere (features_per_slice,)

# Crea il modello di classificazione
classifier = models.Sequential()
classifier.add(layers.InputLayer(input_shape=input_shape))
classifier.add(layers.Dense(256, activation='relu'))
classifier.add(layers.Dropout(0.5))
classifier.add(layers.Dense(1, activation='sigmoid'))  # Sigmoid per la classificazione binaria

# Compila il modello
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Addestra il modello
classifier.fit(X, y, epochs=10, batch_size=2)


NameError: name 'X' is not defined

In [11]:
pip install classification-models-3D

  Obtaining dependency information for classification-models-3D from https://files.pythonhosted.org/packages/63/cc/4929283983829803b6cfd9b24bf79da33f5a8e459f31b653d99283eb78a8/classification_models_3D-1.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 2.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:

from classification_models_3D.kkeras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
model = ResNet18(input_shape=(128, 128, 128, 3), weights='imagenet')




133071032/133071032 ━━━━━━━━━━━━━━━━━━━━ 30s 0us/step


In [14]:
ResNet18, preprocess_input = Classifiers.get('resnet18')
model = ResNet18(
   input_shape=(64, 64, 64, 1),
   stride_size=4,
   kernel_size=3, 
   weights=None
)